In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
os.chdir(os.path.expanduser(
    '~/wcEcoli/models/ecoli/analysis/local_notebooks/C_limited_PDR_analyses_with_published_paper_data/Clim_sorting_combos/Clim0_using_biorxiv_2023_data/CLClim0NE/'))
import plotly.graph_objects as go
import glob
import plotly.express as px

# C-lim half life experimental data (from first Clim sort, using values from the 2023 bioRxiv preprint) 

The data used for Clim rates in this analysis (the first analysis of the Clim rates) comes from this preprint: https://www.biorxiv.org/content/10.1101/2022.08.01.502339v2. The paper has been published, and can be found here: Gupta, et al., 2024: https://www.nature.com/articles/s41467-024-49920-8#Sec20

Specifically, this data comes from Supplementary Table 1: "TableS1 - Half-lives for 3262 proteins across 13 growth conditions >>", which contains the half-lives for proteins in E. coli under carbon-limited conditions. This is the data we are interested in comparing to our model predictions, as the conditions are most similar to our base case.

NOTE that this data has some "ceiling" values it auto assigns proteins to (as defined in the published paper). In the original Clim sort done using the bioarchiv paper (which will be analyzed here), we were not aware that these were ceilinig values. Thus, they are included in this version, but will likely be removed in later Clim rate sorts for half-life value extractions. Also, any protein that did not have a value reported for any one of its doubling time replicates (six total), were also excluded from this sort (leaving roughly 1912 proteins assigned to Clim rates in this CLClimNE sort, 7 assigned to CL, and 2391 to NE).  

ALSO NOTE: even though this file is in the published paper notebook folder, it is still based on the intitial bioRxiv preprint data sort. Future sorts will be done with the new published paper data.


## Find essential genes in the Clim0 data

Doing this by first finding the gene names for each of the monomer ids in the Clim0 data, and then comparing these to the essential genes list from the model.

In [2]:
# FUNCTIONS

# get the gene IDs for each monomer IDs:
def get_gene_ids_for_monomer_ids():
    # code adapted from convert_to_flat.py
    import io
    from wholecell.io import tsv
    from wholecell.utils.filepath import ROOT_PATH
    RNAS_FILE = os.path.join(ROOT_PATH, 'reconstruction', 'ecoli', 'flat', 'rnas.tsv')
    with io.open(RNAS_FILE, 'rb') as f:
        reader = tsv.reader(f, delimiter='\t')
        headers = next(reader)
        while headers[0].startswith('#'):
            headers = next(reader)

        gene_id_index = headers.index('gene_id')
        protein_id_index = headers.index('monomer_ids')
        monomer_ids_to_gene_ids = {}
        for line in reader:
            gene_id = line[gene_id_index]
            protein_id = list(line[protein_id_index][2:-2].split('", "'))[0]
            monomer_ids_to_gene_ids[protein_id] = gene_id

        return monomer_ids_to_gene_ids
    
def get_gene_symbols_for_monomer_ids():
    # code adapted from convert_to_flat.py
    import io
    from wholecell.io import tsv
    from wholecell.utils.filepath import ROOT_PATH
    RNAS_FILE = os.path.join(ROOT_PATH, 'reconstruction', 'ecoli', 'flat', 'rnas.tsv')
    with io.open(RNAS_FILE, 'rb') as f:
        reader = tsv.reader(f, delimiter='\t')
        headers = next(reader)
        while headers[0].startswith('#'):
            headers = next(reader)

        gene_symbol_index = headers.index('common_name')
        protein_id_index = headers.index('monomer_ids')
        monomer_ids_to_gene_symbols = {}
        for line in reader:
            gene_symbol = line[gene_symbol_index]
            protein_id = list(line[protein_id_index][2:-2].split('", "'))[0]
            monomer_ids_to_gene_symbols[protein_id] = gene_symbol

        return monomer_ids_to_gene_symbols
    
    
# convert gene IDs to monomer IDs:
def get_monomer_ids_for_gene_ids():
    # code adapted from convert_to_flat.py
    import io
    from wholecell.io import tsv
    from wholecell.utils.filepath import ROOT_PATH
    RNAS_FILE = os.path.join(ROOT_PATH, 'reconstruction', 'ecoli', 'flat', 'rnas.tsv')
    with io.open(RNAS_FILE, 'rb') as f:
        reader = tsv.reader(f, delimiter='\t')
        headers = next(reader)
        while headers[0].startswith('#'):
            headers = next(reader)

        gene_id_index = headers.index('gene_id')
        protein_id_index = headers.index('monomer_ids')
        gene_ids_to_monomer_ids = {}
        for line in reader:
            gene_id = line[gene_id_index]
            protein_id = list(line[protein_id_index][2:-2].split('", "'))[0]
            gene_ids_to_monomer_ids[gene_id] = protein_id

        return gene_ids_to_monomer_ids
    
    
# recall the genes defined as "essential" in the model, and return them as a list of gene names:
def get_essential_genes():
    # code adapted from convert_to_flat.py
    import io
    from wholecell.io import tsv
    from wholecell.utils.filepath import ROOT_PATH
    GENES_FILE = os.path.join(ROOT_PATH, 'validation', 'ecoli', 'flat', 'essential_genes.tsv')
    with io.open(GENES_FILE, 'rb') as f:
        reader = tsv.reader(f, delimiter='\t')
        headers = next(reader)
        while headers[0].startswith('#'):
            headers = next(reader)

        gene_id_index = headers.index('FrameID')
        essential_genes = []
        for line in reader:
            essential_gene = line[gene_id_index]
            essential_genes.append(essential_gene)

        return essential_genes
    


In [3]:
# load in the Clim0 data:
Clim0_data = pd.read_csv('ParCa_rate_constants/Clim_rate_constants.csv')
monomer_ids = Clim0_data['Protein ID']

Clim0_essential_gene_symbols_found = []
Clim0_essential_monomer_ids_found = []

# todo: figure out how to clean up the run time speed on this function
# find the essential genes in the Clim0 data:
for monomer in monomer_ids:
    # get the gene name for the monomer ID:
    gene_id = get_gene_ids_for_monomer_ids()[monomer]
    if gene_id in get_essential_genes():
        Clim0_essential_gene_symbols_found.append(get_gene_symbols_for_monomer_ids()[monomer])
        Clim0_essential_monomer_ids_found.append(monomer)

print("Essential Genes Found in Clim0 (" +str(len(Clim0_essential_gene_symbols_found))+"): ", Clim0_essential_gene_symbols_found)

# to free up memory:
del Clim0_data, monomer_ids

Essential Genes Found in Clim0 (341):  ['leuA', 'ubiE', 'ubiB', 'panB', 'leuB', 'fabG', 'ubiA', 'bioF', 'argE', 'adk', 'purA', 'cysC', 'purH', 'purM', 'cysJ', 'trpE', 'argS', 'argH', 'argG', 'aroA', 'aroB', 'aroC', 'aroE', 'purB', 'asnS', 'asd', 'pyrB', 'thrA', 'metL', 'aspS', 'atpB', 'atpC', 'hisG', 'accB', 'cysI', 'accC', 'bioB', 'birA', 'ilvE', 'accA', 'accD', 'carB', 'pheA', 'tyrA', 'gltA', 'kdsB', 'pyrG', 'hisF', 'cydA', 'cydC', 'cysN', 'cysS', 'metC', 'bioA', 'lysA', 'dapA', 'folA', 'pyrC', 'pyrD', 'dapB', 'ilvD', 'ribB', 'tmk', 'dut', 'dxr', 'dxs', 'bioH', 'cca', 'lnt', 'dicA', 'dnaB', 'dnaC', 'dnaE', 'dnaN', 'era', 'ffh', 'frr', 'ftsA', 'ftsI', 'ftsY', 'ftsZ', 'fusA', 'ispG', 'grpE', 'gyrA', 'gyrB', 'infA', 'infC', 'lepB', 'ligA', 'map', 'minD', 'minE', 'groS', 'mrdA', 'mrdB', 'mreB', 'mreC', 'msbA', 'mukB', 'nusA', 'nusG', 'parE', 'prfA', 'pth', 'lptE', 'rnc', 'rne', 'rplB', 'rplC', 'rplD', 'rplE', 'rplF', 'rplJ', 'rplL', 'rplM', 'rplN', 'rplO', 'rplP', 'rplQ', 'rplR', 'rplS',

In [4]:
# for my own reference, find the number of essential genes in the the NE and CL groups in CLClim0NE too: 
CL_data = pd.read_csv('ParCa_rate_constants/CL_rate_constants.csv')
CL_monomer_ids = CL_data['Protein ID']

CL_essential_gene_symbols_found = []
CL_essential_monomer_ids_found = []
for monomer in CL_monomer_ids:
    # get the gene name for the monomer ID:
    gene_id = get_gene_ids_for_monomer_ids()[monomer]
    if gene_id in get_essential_genes():
        CL_essential_gene_symbols_found.append(get_gene_symbols_for_monomer_ids()[monomer])
        CL_essential_monomer_ids_found.append(monomer)

print("Essential Genes Found in CL (" +str(len(CL_essential_gene_symbols_found))+"): ", CL_essential_gene_symbols_found)

NE_data = pd.read_csv('ParCa_rate_constants/NE_rate_constants.csv')
NE_monomer_ids = NE_data['Protein ID']

NE_essential_gene_symbols_found = []
NE_essential_monomer_ids_found = []
for monomer in NE_monomer_ids:
    # get the gene name for the monomer ID:
    gene_id = get_gene_ids_for_monomer_ids()[monomer]
    if gene_id in get_essential_genes():
        NE_essential_gene_symbols_found.append(get_gene_symbols_for_monomer_ids()[monomer])
        NE_essential_monomer_ids_found.append(monomer)
        
print("Essential Genes Found in NE (" +str(len(NE_essential_gene_symbols_found))+"): ", NE_essential_gene_symbols_found)

# to free up memory:
del CL_data, NE_data, CL_monomer_ids, NE_monomer_ids

Essential Genes Found in CL (3):  ['carA', 'cdsA', 'bioD']
Essential Genes Found in NE (60):  ['plsC', 'trpD', 'panD', 'yagG', 'dfp', 'bioC', 'ftsQ', 'ftsW', 'lspA', 'mazE', 'mreD', 'rnpA', 'rpmC', 'rpmH', 'ftsL', 'secM', 'yihA', 'chpS', 'leuL', 'ydfB', 'tadA', 'holB', 'ispF', 'alsK', 'lapC', 'lgt', 'yhhQ', 'prmC', 'yabQ', 'yefM', 'entD', 'wzyE', 'murJ', 'yceQ', 'ymfK', 'racR', 'ydiL', 'exoX', 'zipA', 'ftsB', 'yqgD', 'mqsA', 'tdcF', 'ubiV', 'igaA', 'lptF', 'hemA', 'folK', 'metA', 'nadK', 'pabA', 'coaD', 'mraY', 'pgsA', 'rpoH', 'secE', 'lpxK', 'tnaB', 'ispU', 'lolD']


In [5]:
# I would like to see if Lon or Clp are in this out of curiosity:
essential_gene_symbols = []
for gene in get_essential_genes():
    essential_gene_symbols.append(get_gene_symbols_for_monomer_ids()[get_monomer_ids_for_gene_ids()[gene]])
    
print("Essential Genes in Model (" +str(len(essential_gene_symbols))+"): ", essential_gene_symbols)

del essential_gene_symbols

Essential Genes in Model (406):  ['argG', 'bioA', 'bioB', 'bioC', 'bioD', 'bioF', 'bioH', 'ilvE', 'cysG', 'ubiE', 'exoX', 'hflD', 'argA', 'argE', 'argH', 'aroA', 'aroB', 'aroC', 'aroE', 'cysC', 'cysE', 'cysH', 'cysI', 'cysJ', 'cysN', 'glyA', 'hisA', 'hisB', 'hisC', 'hisD', 'hisF', 'hisG', 'hisH', 'hisI', 'ilvA', 'ilvC', 'ilvD', 'leuA', 'leuB', 'leuC', 'leuD', 'lysA', 'metA', 'metB', 'metC', 'metF', 'metL', 'pabA', 'pabB', 'pheA', 'proA', 'proB', 'proC', 'serA', 'serB', 'serC', 'thrA', 'thrB', 'thrC', 'trpA', 'trpB', 'trpC', 'trpD', 'trpE', 'tyrA', 'carA', 'carB', 'guaA', 'guaB', 'purA', 'purC', 'purD', 'purE', 'purF', 'purH', 'purK', 'purL', 'purM', 'pyrB', 'pyrC', 'pyrD', 'pyrE', 'pyrF', 'thyA', 'iscS', 'nadA', 'nadB', 'nadC', 'panB', 'panC', 'panD', 'pdxA', 'pdxB', 'pdxH', 'pdxJ', 'cysB', 'leuL', 'metR', 'ptsI', 'atpB', 'atpC', 'glnA', 'gltA', 'icd', 'ppc', 'panZ', 'ribF', 'lspA', 'ispH', 'dapB', 'folA', 'lptD', 'yabQ', 'ftsL', 'ftsI', 'murE', 'murF', 'mraY', 'murD', 'ftsW', 'murG', 

# Determining which proteins are assigned to a protease (according to Gupta et al., 2024) and are essential in the CLClim0NE sorted model

This data is from a rendition of supplementary table #2 in the published paper ("TableS2 - Assignment of protein substrates to proteases >>"). This table contains the proteins that are assigned to each of the proteases in the proteins they measured in the paper. This supplementary table has been adapted for use in this analysis, so the full detail about each data column is not included here (but can be found in the original paper). 


In [6]:
# load in the protease-substrate assignment datatable: 
protease_data = pd.read_excel('~/wcEcoli/models/ecoli/analysis/local_notebooks/C_limited_PDR_analyses_with_published_paper_data/supplementary_data/Gupta_et_al_2024_ST2_protease_assignment_data.xlsx')

# determine if any of the proteins assigned to a protease are considered an essential gene in the model: 
essential_substrate_gene_symbols = []
for gene in get_essential_genes():
    if get_gene_symbols_for_monomer_ids()[get_monomer_ids_for_gene_ids()[gene]] in protease_data['Gene name'].values:
        essential_substrate_gene_symbols.append(get_gene_symbols_for_monomer_ids()[get_monomer_ids_for_gene_ids()[gene]])
        
print("There are " + str(len(protease_data['Gene name'].values)) + " proteins assigned to a protease in the Gupta et al., 2024 data.")
print("Essential Genes in Model that are Assigned to a Protease (" +str(len(essential_substrate_gene_symbols))+"): ", essential_substrate_gene_symbols)

There are 308 proteins assigned to a protease in the Gupta et al., 2024 data.
Essential Genes in Model that are Assigned to a Protease (57):  ['bioB', 'ilvE', 'argA', 'aroE', 'cysH', 'cysI', 'cysJ', 'cysN', 'leuA', 'leuC', 'leuD', 'lysA', 'thrA', 'trpB', 'tyrA', 'iscS', 'nadA', 'nadB', 'metR', 'ispH', 'ftsI', 'ftsQ', 'ftsZ', 'lpxC', 'erpA', 'frr', 'dxr', 'ispU', 'dnaE', 'thiL', 'dnaX', 'ftsK', 'rpsA', 'mukB', 'acpP', 'minE', 'hemA', 'folE', 'ligA', 'dapA', 'ispG', 'era', 'ffh', 'metK', 'parE', 'ribB', 'obgE', 'def', 'rplX', 'dnaA', 'birA', 'rpoB', 'rpoC', 'ubiA', 'lexA', 'dnaB', 'dnaC']


In [7]:
# determine which proteins are assigned to proteases in the Clim0 data (does not matter if they are essential or not):
Clim0_data = pd.read_csv('ParCa_rate_constants/Clim_rate_constants.csv')
monomer_ids = Clim0_data['Protein ID']

Clim0_protease_substrate_gene_symbols = []
Clim0_protease_substrate_monomer_ids = []
for monomer in monomer_ids:
    # get the gene symbol for the monomer ID:
    gene_symbol = get_gene_symbols_for_monomer_ids()[monomer]
    if gene_symbol in protease_data['Gene name'].values:
        Clim0_protease_substrate_gene_symbols.append(gene_symbol)
        Clim0_protease_substrate_monomer_ids.append(monomer)
        
print("Total Protease Substrates in Clim0 (" +str(len(Clim0_protease_substrate_gene_symbols))+"): ", Clim0_protease_substrate_gene_symbols)

# to free up memory:
del Clim0_data, monomer_ids

# repeat for CL
CL_data = pd.read_csv('ParCa_rate_constants/CL_rate_constants.csv')
CL_monomer_ids = CL_data['Protein ID']

CL_protease_substrate_gene_symbols = []
CL_protease_substrate_monomer_ids = []
for monomer in CL_monomer_ids:
    # get the gene symbol for the monomer ID:
    gene_symbol = get_gene_symbols_for_monomer_ids()[monomer]
    if gene_symbol in protease_data['Gene name'].values:
        CL_protease_substrate_gene_symbols.append(gene_symbol)
        CL_protease_substrate_monomer_ids.append(monomer)
        
print("Total Protease Substrates in CL (" +str(len(CL_protease_substrate_gene_symbols))+"): ", CL_protease_substrate_gene_symbols)

# to free up memory:
del CL_data, CL_monomer_ids

NE_data = pd.read_csv('ParCa_rate_constants/NE_rate_constants.csv')
NE_monomer_ids = NE_data['Protein ID']

NE_protease_substrate_gene_symbols = []
NE_protease_substrate_monomer_ids = []
for monomer in NE_monomer_ids:
    # get the gene symbol for the monomer ID:
    gene_symbol = get_gene_symbols_for_monomer_ids()[monomer]
    if gene_symbol in protease_data['Gene name'].values:
        NE_protease_substrate_gene_symbols.append(gene_symbol)
        NE_protease_substrate_monomer_ids.append(monomer)
        
print("Total Protease Substrates in NE (" +str(len(NE_protease_substrate_gene_symbols))+"): ", NE_protease_substrate_gene_symbols)

# to free up memory:
del NE_data, NE_monomer_ids

# note that for CLClim0NE, the breakdown appears to be as follows: Clim0 (273), CL (0), NE (25). There are 308 proteins assigned to proteases in the Gupta et al., 2024 data, and 298 of these appear in the whole cell model.

Total Protease Substrates in Clim0 (273):  ['leuA', 'ubiA', 'ackA', 'acnB', 'fadE', 'glpD', 'cysJ', 'uxaA', 'astA', 'aroD', 'aroE', 'aroG', 'aroK', 'aroL', 'asnA', 'aspA', 'thrA', 'cysI', 'bioB', 'birA', 'ilvE', 'chbB', 'cfa', 'tyrA', 'ubiC', 'cycA', 'cysD', 'cysN', 'dadA', 'lysA', 'dapA', 'nfsB', 'ribB', 'cobT', 'dxr', 'yaaA', 'bisC', 'clpA', 'clpX', 'dedA', 'dedD', 'dksA', 'dnaB', 'dnaC', 'dnaE', 'dnaK', 'dnaQ', 'era', 'ffh', 'frr', 'ftsI', 'ftsZ', 'ispG', 'mnmG', 'greA', 'helD', 'hflX', 'hmp', 'hsdM', 'hypE', 'ligA', 'mazG', 'minE', 'mukB', 'mutS', 'nfo', 'parC', 'parE', 'pcnB', 'priC', 'pspA', 'recA', 'recD', 'recF', 'recJ', 'rhlB', 'rplX', 'rpsA', 'ruvC', 'sbcC', 'selD', 'prlF', 'srmB', 'tonB', 'topB', 'uvrB', 'uvrD', 'ispH', 'sbcD', 'ybaB', 'relE', 'yebC', 'yecA', 'uvrY', 'yibA', 'yidA', 'mioC', 'ybhA', 'rlmD', 'mazF', 'radA', 'rng', 'lipA', 'dinG', 'ahpC', 'ahpF', 'ychF', 'yjgA', 'dps', 'def', 'yigI', 'ycgB', 'ibpA', 'greB', 'grcA', 'yihD', 'yihI', 'yiiQ', 'rsuA', 'radD', 'ccmH'

In [8]:
NE_protease_substrate_monomer_ids

['ADENYLATECYC-MONOMER',
 'ADHP-MONOMER',
 'EG10125-MONOMER',
 'EG10229-MONOMER',
 'EG10342-MONOMER',
 'EG10724-MONOMER',
 'EG10986-MONOMER',
 'EG11089-MONOMER',
 'EG11544-MONOMER',
 'EG11734-MONOMER',
 'EG11783-MONOMER',
 'EG11889-MONOMER',
 'EG12069-MONOMER',
 'EG12402-MONOMER',
 'G6460-MONOMER',
 'G6523-MONOMER',
 'G6670-MONOMER',
 'G6994-MONOMER',
 'G7657-MONOMER',
 'GLUTRNAREDUCT-MONOMER',
 'MDLB-MONOMER',
 'MONOMER0-741',
 'NARW-MONOMER',
 'TREHALACYTO-MONOMER',
 'UPPSYN-MONOMER']

In [9]:
# check if any of the essential genes in the Clim0 data are assigned to a protease:
essential_substrate_gene_symbols = []
for gene in Clim0_essential_gene_symbols_found:
    if gene in protease_data['Gene name'].values:
        essential_substrate_gene_symbols.append(gene)
        
print("Essential Genes with Clim0 half-lives that are assigned to a protease (" +str(len(essential_substrate_gene_symbols))+"): ", essential_substrate_gene_symbols)

Essential Genes with Clim0 half-lives that are assigned to a protease (54):  ['leuA', 'ubiA', 'cysJ', 'aroE', 'thrA', 'cysI', 'bioB', 'birA', 'ilvE', 'tyrA', 'cysN', 'lysA', 'dapA', 'ribB', 'dxr', 'dnaB', 'dnaC', 'dnaE', 'era', 'ffh', 'frr', 'ftsI', 'ftsZ', 'ispG', 'ligA', 'minE', 'mukB', 'parE', 'rplX', 'rpsA', 'ispH', 'def', 'erpA', 'acpP', 'ftsK', 'iscS', 'obgE', 'folE', 'nadB', 'leuC', 'leuD', 'dnaX', 'argA', 'cysH', 'lexA', 'dnaA', 'metR', 'nadA', 'rpoB', 'rpoC', 'metK', 'thiL', 'trpB', 'lpxC']


In [10]:
# check if any of the essential genes in the CL data are assigned to a protease:
CL_essential_substrate_gene_symbols = []
for gene in CL_essential_gene_symbols_found:
    if gene in protease_data['Gene name'].values:
        CL_essential_substrate_gene_symbols.append(gene)
        
print("Essential Genes with CL half-lives that are assigned to a protease (" +str(len(CL_essential_substrate_gene_symbols))+"): ", CL_essential_substrate_gene_symbols)

# check if any of the essential genes in the NE data are assigned to a protease:
NE_essential_substrate_gene_symbols = []
for gene in NE_essential_gene_symbols_found:
    if gene in protease_data['Gene name'].values:
        NE_essential_substrate_gene_symbols.append(gene)
        
print("Essential Genes with NE half-lives that are assigned to a protease (" +str(len(NE_essential_substrate_gene_symbols))+"): ", NE_essential_substrate_gene_symbols)

Essential Genes with CL half-lives that are assigned to a protease (0):  []
Essential Genes with NE half-lives that are assigned to a protease (3):  ['ftsQ', 'hemA', 'ispU']


next items to do with this info: 
- plot the essential genes in the Clim0 data
- plot the outliers and highlight the essential genes 
- plot the outliers and highlight the protease substrates
- plot of the outliers and highlight the type of protease substrate

# Plot the essential genes in CLClim0NE

In [30]:
# FUNCTIONS:

# convert the data from the ParCa translation.py raw HL saves: 
def convert_HL_data(dataframe):
    name_s = 'rate constant (s^1)' # name in seconds
    name_m = 'half life (min)' # name in mins
    dataframe.rename(columns={'Rate Constant': name_s}, inplace=True)
    dataframe[name_m] = dataframe[name_s] * 60 * (1/np.log(2))
    dataframe[name_m] = 1 / dataframe[name_m] # units of mins
    return dataframe


# convert each data source in the saved HL data file folders 
def convert_data_and_add_HL_source(dataframe, HL_source):
    name_s = HL_source + ' rate constant (s^1)' # name in seconds
    name_m = HL_source + ' half life (min)' # name in mins
    dataframe.rename(columns={'Rate Constant': name_s}, inplace=True)
    dataframe[name_m] = dataframe[name_s] * 60 * (1/np.log(2))
    dataframe[name_m] = 1 / dataframe[name_m] # units of mins
    return dataframe

# CLMLNE_ML_df = convert_data(CLMLNE_ML, "CLMLNE ML")
# CLMLNE_CL_df = convert_data(CLMLNE_CL, "CLMLNE CL")
# CLMLNE_NE_df = convert_data(CLMLNE_NE, "CLMLNE NE")
# CLNE_CL_df = convert_data(CLNE_CL, "CLNE CL")
# CLNE_NE_df = convert_data(CLNE_NE, "CLNE NE")

In [12]:
# determine which proteins are essential genes in the Clim0 data (does not matter if they are assigned to proteases or not):
Clim0_data = pd.read_csv('ParCa_rate_constants/Clim_rate_constants.csv')
Clim0_data['HL Source'] = "Clim0" # add a column with the half life source
monomer_ids = Clim0_data['Protein ID']

# if a monomer in Clim0_essential_monomer_ids_found is found in monomer_ids, remove them from Clim0_data and add it to a new dataframe: 
essential_monomer_idxs = monomer_ids.isin(Clim0_essential_monomer_ids_found)
Clim0_essential_monomers = Clim0_data[essential_monomer_idxs].copy()
Clim0_essential_monomers['HL Source'] = "Clim0 (essential gene)"
Clim0_data_remaining = Clim0_data[~essential_monomer_idxs].copy()

#del Clim0_data, monomer_ids

# repeat for CL: 
CL_data = pd.read_csv('ParCa_rate_constants/CL_rate_constants.csv')
CL_data['HL Source'] = "CL" # add a column with the half life source
CL_monomer_ids = CL_data['Protein ID']

# if a monomer in CL_essential_monomer_ids_found is found in monomer_ids, remove them from Clim0_data and add it to a new dataframe: 
CL_essential_monomer_idxs = CL_monomer_ids.isin(CL_essential_monomer_ids_found)
CL_essential_monomers = CL_data[CL_essential_monomer_idxs].copy()
CL_essential_monomers['HL Source'] = "CL (essential gene)"
CL_data_remaining = CL_data[~CL_essential_monomer_idxs].copy()
        
#del CL_data, CL_monomer_ids

# repeat for NE: 
NE_data = pd.read_csv('ParCa_rate_constants/NE_rate_constants.csv')
NE_data['HL Source'] = "NE" # add a column with the half life source
NE_monomer_ids = NE_data['Protein ID']

# if a monomer in NE_essential_monomer_ids_found is found in monomer_ids, remove them from Clim0_data and add it to a new dataframe: 
NE_essential_monomer_idxs = NE_monomer_ids.isin(NE_essential_monomer_ids_found)
NE_essential_monomers = NE_data[NE_essential_monomer_idxs].copy()
NE_essential_monomers['HL Source'] = "NE (essential gene)"
NE_data_remaining = NE_data[~NE_essential_monomer_idxs].copy()

#del NE_data, NE_monomer_ids


In [14]:
# USER INPUTS: define the name of the combo in use and the special Clim name
current_sequence = "CLClim0NE"
Clim_name = 'Clim0'

Plot #1: A plot of  all the protein half lifes in the combo (no specific order, but sorted by source)

In [15]:
# Plot the half-lifes of CLClim0NE in order
CLClim0NE_df_full = pd.concat([CL_data, Clim0_data, NE_data], ignore_index=True) # concatinate row-wise

# convert the units of the data: 
CLClim0NE_df_full = convert_HL_data(CLClim0NE_df_full)

# sort the rates first:
#CLClim0NE_df_full = CLClim0NE_df_full.sort_values(by=["half life (min)"], ascending=True)
fig = px.scatter(CLClim0NE_df_full, x="Protein ID", y=CLClim0NE_df_full["half life (min)"], color="HL Source")
fig.update_traces(marker=dict(size=3, ), )

# plot specs:
fig.update_xaxes(visible=False)
fig.update_layout(title='The Half Life Values for Proteins in the ' + current_sequence + ' Sorted Combo<br>', xaxis_title='Protein ID', yaxis_title='Half Life (min)')
fig.update_layout(legend_title_text="Half Life Source")

# wont open in pycharm, so save as a html:
out_pth = "~/wcEcoli/out/random_plotlys/" + current_sequence + "_partitioned.html"
out_pth = os.path.expanduser(out_pth)  
output_dir = os.path.dirname(out_pth)
os.makedirs(output_dir, exist_ok=True)  
fig.write_html(out_pth, auto_open=True)

Plot #2: A plot of the proteins' half life assignment, ordered by value

In [16]:
# plot in order! (plotly does not naturally plot in order while merging over the different subsets globally)
CLClim0NE_df_full['gene symbol'] = CLClim0NE_df_full['Protein ID'].apply(lambda x: get_gene_symbols_for_monomer_ids()[x]) # this is to add to the hovertext
ordered_df = CLClim0NE_df_full.sort_values(by=["half life (min)"], ascending=True)

# create a dataframe for the sources of different rates: 
ordered_CL_df = ordered_df[ordered_df['HL Source'] == "CL"]
ordered_Clim_df = ordered_df[ordered_df['HL Source'] == Clim_name]
ordered_NE_df = ordered_df[ordered_df['HL Source'] == "NE"]

# generate the figure in the background first:
fig = px.scatter(ordered_df, x="Protein ID", y=CLClim0NE_df_full["half life (min)"], )
fig.update_traces(marker_size=.5, opacity=.3)

# overlay the individual rates
fig.add_trace(go.Scatter(x=(ordered_Clim_df['Protein ID']), y=(ordered_Clim_df["half life (min)"]), mode='markers', name=Clim_name, hovertext=ordered_Clim_df['gene symbol'], marker_size=5, marker=dict(color="darkorange")))
fig.add_trace(go.Scatter(x=(ordered_NE_df['Protein ID']), y=(ordered_NE_df["half life (min)"]), mode='markers', name='NE', hovertext=ordered_NE_df['gene symbol'], marker_size=5, marker=dict(color="lightseagreen")))
fig.add_trace(go.Scatter(x=(ordered_CL_df['Protein ID']), y=(ordered_CL_df["half life (min)"]), mode='markers', name='CL', hovertext=ordered_CL_df['gene symbol'], marker_size=7, marker=dict(color="deeppink"))) # putting this last so that I can see it stacked on top of the others

# plot specs 
fig.update_xaxes(visible=False)
fig.update_layout(legend_title_text='Half Life Source')
fig.update_layout(title='The Half Life Values for Proteins in the ' + current_sequence + ' Sorted Combo<br>', xaxis_title='Protein ID', yaxis_title='Half Life (min)')

# wont open in pycharm, so save as a html:
out_pth = "~/wcEcoli/out/random_plotlys/" + current_sequence + "_ordered.html"
out_pth = os.path.expanduser(out_pth) 
output_dir = os.path.dirname(out_pth)
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it does not exist
fig.write_html(out_pth, auto_open=True) # displays the figure on a default web brouser 

Plot #3: Graph of the proteins with their essential genes highlighted

In [17]:
# combine all the data frames
dfs = [Clim0_essential_monomers, Clim0_data_remaining, CL_essential_monomers, CL_data_remaining, NE_essential_monomers, NE_data_remaining]
CLClim0NE_df_full_withEGs = pd.concat(dfs, ignore_index=True) # concatinate row-wise

# plot in order! (plotly does not naturally plot in order while merging over the different subsets globally)
CLClim0NE_df_full_withEGs = convert_HL_data(CLClim0NE_df_full_withEGs)
CLClim0NE_df_full_withEGs['gene symbol'] = CLClim0NE_df_full_withEGs['Protein ID'].apply(lambda x: get_gene_symbols_for_monomer_ids()[x]) # this is to add to the hovertext
ordered_df = CLClim0NE_df_full_withEGs.sort_values(by=["half life (min)"], ascending=True)

# create a dataframe for the sources of different rates: 
ordered_CL_df = ordered_df[ordered_df['HL Source'] == "CL"]
ordered_Clim_df = ordered_df[ordered_df['HL Source'] == Clim_name]
ordered_NE_df = ordered_df[ordered_df['HL Source'] == "NE"]
ordered_CL_EG_df = ordered_df[ordered_df['HL Source'] == "CL (essential gene)"]
ordered_Clim_EG_df = ordered_df[ordered_df['HL Source'] == Clim_name + " (essential gene)"]
ordered_NE_EG_df = ordered_df[ordered_df['HL Source'] == "NE (essential gene)"]

# generate the figure in the background first:
fig = px.scatter(ordered_df, x="Protein ID", y=CLClim0NE_df_full_withEGs["half life (min)"])
fig.update_traces(marker_size=.5, opacity=.1)

# overlay the individual rates
fig.add_trace(go.Scatter(x=(ordered_Clim_df['Protein ID']), y=(ordered_Clim_df["half life (min)"]), mode='markers', name=(Clim_name + ', '+ (str(np.shape(ordered_Clim_df['Protein ID'])[0]))), hovertext=ordered_Clim_df['gene symbol'], marker_size=5, marker=dict(color="darkorange")))
fig.add_trace(go.Scatter(x=(ordered_NE_df['Protein ID']), y=(ordered_NE_df["half life (min)"]), mode='markers', name=('NE, '+ (str(np.shape(ordered_NE_df['Protein ID'])[0]))), hovertext=ordered_NE_df['gene symbol'], marker_size=5, marker=dict(color="lightseagreen")))
fig.add_trace(go.Scatter(x=(ordered_CL_df['Protein ID']), y=(ordered_CL_df["half life (min)"]), mode='markers', name=('CL, ' + (str(np.shape(ordered_CL_df['Protein ID'])[0]))), hovertext=ordered_CL_df['gene symbol'], marker_size=7, marker=dict(color="deeppink"))) 

fig.add_trace(go.Scatter(x=(ordered_Clim_EG_df['Protein ID']), y=(ordered_Clim_EG_df["half life (min)"]), mode='markers', name=(Clim_name + ' (essential genes), ' + (str(np.shape(ordered_Clim_EG_df['Protein ID'])[0]))), hovertext=ordered_Clim_EG_df['gene symbol'], marker_size=3, marker=dict(color="lime", symbol='diamond')))
fig.add_trace(go.Scatter(x=(ordered_NE_EG_df['Protein ID']), y=(ordered_NE_EG_df["half life (min)"]), mode='markers', name=('NE (essential genes), ' + (str(np.shape(ordered_NE_EG_df['Protein ID'])[0]))), hovertext=ordered_NE_EG_df['gene symbol'], marker_size=3, marker=dict(color="yellow", symbol='star')))
fig.add_trace(go.Scatter(x=(ordered_CL_EG_df['Protein ID']), y=(ordered_CL_EG_df["half life (min)"]), mode='markers', name=('CL (essential genes), ' + (str(np.shape(ordered_CL_EG_df['Protein ID'])[0]))), hovertext=ordered_CL_EG_df['gene symbol'], marker_size=3, marker=dict(color="deepskyblue", symbol='square'))) # putting this last so that I can see it stacked on top of the others


# plot specs 
fig.update_xaxes(visible=False)
fig.update_layout(legend_title_text='Half Life Source')
fig.update_layout(title='The Half Life Values for Proteins in the ' + current_sequence + ' Sorted Combo<br>', xaxis_title='Protein ID', yaxis_title='Half Life (min)')

# wont open in pycharm, so save as a html:
out_pth = "~/wcEcoli/out/random_plotlys/" + current_sequence + "_ordered_with_essential_genes.html"
out_pth = os.path.expanduser(out_pth) 
output_dir = os.path.dirname(out_pth)
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it does not exist
fig.write_html(out_pth, auto_open=True) # displays the figure on a default web brouser 

# Plots with the proteins assigned to proteases

In [18]:
# first, clear all the data above, so that I can make the same plots with the same variables
del essential_monomer_idxs, Clim0_essential_monomers, Clim0_data_remaining, CL_essential_monomer_idxs, CL_essential_monomers, CL_data_remaining, NE_essential_monomer_idxs, NE_essential_monomers, NE_data_remaining

In [33]:
# determine which proteins are assigned to proteases in the Clim0 data (does not matter if they are essential or not):
Clim0_data = pd.read_csv('ParCa_rate_constants/Clim_rate_constants.csv')
Clim0_data['HL Source'] = "Clim0" # add a column with the half life source
monomer_ids = Clim0_data['Protein ID']

# if a monomer in Clim0_essential_monomer_ids_found is found in monomer_ids, remove them from Clim0_data and add it to a new dataframe: 
essential_monomer_idxs = monomer_ids.isin(Clim0_protease_substrate_monomer_ids)
Clim0_essential_monomers = Clim0_data[essential_monomer_idxs].copy()
Clim0_essential_monomers['HL Source'] = "Clim0 (protease substrate)"
Clim0_data_remaining = Clim0_data[~essential_monomer_idxs].copy()

#del Clim0_data, monomer_ids

# repeat for CL: 
CL_data = pd.read_csv('ParCa_rate_constants/CL_rate_constants.csv')
CL_data['HL Source'] = "CL" # add a column with the half life source
CL_monomer_ids = CL_data['Protein ID']

# if a monomer in CL_essential_monomer_ids_found is found in monomer_ids, remove them from Clim0_data and add it to a new dataframe: 
CL_essential_monomer_idxs = CL_monomer_ids.isin(CL_protease_substrate_monomer_ids)
CL_essential_monomers = CL_data[CL_essential_monomer_idxs].copy()
CL_essential_monomers['HL Source'] = "CL (protease substrate)"
CL_data_remaining = CL_data[~CL_essential_monomer_idxs].copy()
        
#del CL_data, CL_monomer_ids

# repeat for NE: 
NE_data = pd.read_csv('ParCa_rate_constants/NE_rate_constants.csv')
NE_data['HL Source'] = "NE" # add a column with the half life source
NE_monomer_ids = NE_data['Protein ID']

# if a monomer in NE_essential_monomer_ids_found is found in monomer_ids, remove them from Clim0_data and add it to a new dataframe: 
NE_essential_monomer_idxs = NE_monomer_ids.isin(NE_protease_substrate_monomer_ids)
NE_essential_monomers = NE_data[NE_essential_monomer_idxs].copy()
NE_essential_monomers['HL Source'] = "NE (protease substrate)"
NE_data_remaining = NE_data[~NE_essential_monomer_idxs].copy()

#del NE_data, NE_monomer_ids


Plot #4: Similar to plot 3, but instead of the essential genes plotted, the proteins assigned to substrates are plotted!

In [23]:
# combine all the data frames
dfs = [Clim0_essential_monomers, Clim0_data_remaining, CL_essential_monomers, CL_data_remaining, NE_essential_monomers, NE_data_remaining]
CLClim0NE_df_full_withPSs = pd.concat(dfs, ignore_index=True) # concatinate row-wise

# plot in order! (plotly does not naturally plot in order while merging over the different subsets globally)
CLClim0NE_df_full_withPSs = convert_HL_data(CLClim0NE_df_full_withPSs)
CLClim0NE_df_full_withPSs['gene symbol'] = CLClim0NE_df_full_withPSs['Protein ID'].apply(lambda x: get_gene_symbols_for_monomer_ids()[x]) # this is to add to the hovertext
ordered_df = CLClim0NE_df_full_withPSs.sort_values(by=["half life (min)"], ascending=True)

# create a dataframe for the sources of different rates: 
ordered_CL_df = ordered_df[ordered_df['HL Source'] == "CL"]
ordered_Clim_df = ordered_df[ordered_df['HL Source'] == Clim_name]
ordered_NE_df = ordered_df[ordered_df['HL Source'] == "NE"]
ordered_CL_PS_df = ordered_df[ordered_df['HL Source'] == "CL (protease substrate)"]
ordered_Clim_PS_df = ordered_df[ordered_df['HL Source'] == Clim_name + " (protease substrate)"]
ordered_NE_PS_df = ordered_df[ordered_df['HL Source'] == "NE (protease substrate)"]

# generate the figure in the background first:
fig = px.scatter(ordered_df, x="Protein ID", y=CLClim0NE_df_full_withPSs["half life (min)"])
fig.update_traces(marker_size=.5, opacity=.1)

# overlay the individual rates
fig.add_trace(go.Scatter(x=(ordered_Clim_df['Protein ID']), y=(ordered_Clim_df["half life (min)"]), mode='markers', name=(Clim_name + ', '+ (str(np.shape(ordered_Clim_df['Protein ID'])[0]))), hovertext=ordered_Clim_df['gene symbol'], marker_size=5, marker=dict(color="darkorange")))
fig.add_trace(go.Scatter(x=(ordered_NE_df['Protein ID']), y=(ordered_NE_df["half life (min)"]), mode='markers', name=('NE, '+ (str(np.shape(ordered_NE_df['Protein ID'])[0]))), hovertext=ordered_NE_df['gene symbol'], marker_size=5, marker=dict(color="lightseagreen")))
fig.add_trace(go.Scatter(x=(ordered_CL_df['Protein ID']), y=(ordered_CL_df["half life (min)"]), mode='markers', name=('CL, ' + (str(np.shape(ordered_CL_df['Protein ID'])[0]))), hovertext=ordered_CL_df['gene symbol'], marker_size=7, marker=dict(color="deeppink"))) 

fig.add_trace(go.Scatter(x=(ordered_Clim_PS_df['Protein ID']), y=(ordered_Clim_PS_df["half life (min)"]), mode='markers', name=(Clim_name + ' (protease substrates), ' + (str(np.shape(ordered_Clim_PS_df['Protein ID'])[0]))), hovertext=ordered_Clim_PS_df['gene symbol'], marker_size=3, marker=dict(color="lime", symbol='diamond')))
fig.add_trace(go.Scatter(x=(ordered_NE_PS_df['Protein ID']), y=(ordered_NE_PS_df["half life (min)"]), mode='markers', name=('NE (protease substrates), ' + (str(np.shape(ordered_NE_PS_df['Protein ID'])[0]))), hovertext=ordered_NE_PS_df['gene symbol'], marker_size=3, marker=dict(color="yellow", symbol='star')))
fig.add_trace(go.Scatter(x=(ordered_CL_PS_df['Protein ID']), y=(ordered_CL_PS_df["half life (min)"]), mode='markers', name=('CL (protease substrates), ' + (str(np.shape(ordered_CL_PS_df['Protein ID'])[0]))), hovertext=ordered_CL_PS_df['gene symbol'], marker_size=3, marker=dict(color="deepskyblue", symbol='square'))) # putting this last so that I can see it stacked on top of the others


# plot specs 
fig.update_xaxes(visible=False)
fig.update_layout(legend_title_text='Half Life Source')
fig.update_layout(title='The Half Life Values for Proteins in the ' + current_sequence + ' Sorted Combo<br>', xaxis_title='Protein ID', yaxis_title='Half Life (min)')

# wont open in pycharm, so save as a html:
out_pth = "~/wcEcoli/out/random_plotlys/" + current_sequence + "_ordered_with_protein_substrates.html"
out_pth = os.path.expanduser(out_pth) 
output_dir = os.path.dirname(out_pth)
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it does not exist
fig.write_html(out_pth, auto_open=True) # displays the figure on a default web brouser 

Plot #5: a plot of just the Clim0 half-lifes and the ones that are essential 

In [80]:
# first, prepare the data for merging:
del Clim0_data, monomer_ids, Clim0_data_remaining

# separate CLim0 data
Clim0_data = pd.read_csv('ParCa_rate_constants/Clim_rate_constants.csv')
Clim0_data['HL Source'] = "Clim0" # add a column with the half life source
monomer_ids = Clim0_data["Protein ID"]

# if a monomer in Clim0_protease_substrate_monomer_ids is found in monomer_ids, remove them from Clim0_data and add it to a new dataframe: 
Clim0_protease_substrate_idxs = monomer_ids.isin(Clim0_protease_substrate_monomer_ids)
Clim0_protease_substrates = Clim0_data[Clim0_protease_substrate_idxs].copy()
Clim0_protease_substrates['HL Source'] = "Clim0 (protease substrate)"
Clim0_data_remaining = Clim0_data[~Clim0_protease_substrate_idxs].copy()

# merge it back together momentarily: 
Clim0_data = pd.concat([Clim0_protease_substrates, Clim0_data_remaining], ignore_index=True)

# load in the "N-end" rule rates from the CLNE model (CL is the exact same between both):
CLNE_NE_data = pd.read_excel('~/wcEcoli/models/ecoli/analysis/local_notebooks/C_limited_PDR_analyses_with_published_paper_data/previous_models/CLNE_files/CLNE_full.xlsx')
CLNE_NE_data.rename(columns={CLNE_NE_data.columns[1]: "Rate Constant"}, inplace=True) 

# convert both datasets: 
CLNE_NE_data = convert_data_and_add_HL_source(CLNE_NE_data, "CLNE NE")
CLClim0NE_Clim0_data = convert_data_and_add_HL_source(Clim0_data, "CLClim0NE Clim0")

In [88]:
# second, merge the data: 

# merge the two dataframes on the protein names
df = pd.merge(CLClim0NE_Clim0_data, CLNE_NE_data, on='Protein ID', how='inner')

# make a new data frame with all proteins that have a half life of greater than 10 mins in the NE half life (min) column 
df_10hrs = df[df['CLNE NE half life (min)'] > 10].copy()
monomer_ids_10hrs = df_10hrs['Protein ID'] # todo: do I need this??
df_10hrs['gene symbol'] = df_10hrs['Protein ID'].apply(lambda x: get_gene_symbols_for_monomer_ids()[x])
df_10hrs = df_10hrs.drop(columns=["CLNE NE rate constant (s^1)", "CLNE NE half life (min)", "CLClim0NE Clim0 rate constant (s^1)"])
df_10hrs

,Protein ID,HL Source,CLClim0NE Clim0 half life (min),gene symbol
0,2-ISOPROPYLMALATESYN-MONOMER,Clim0 (protease substrate),222.8,leuA
1,4OHBENZOATE-OCTAPRENYLTRANSFER-MONOMER,Clim0 (protease substrate),76.5,ubiA
2,ACETATEKINA-MONOMER,Clim0 (protease substrate),720.0,ackA
3,ACONITATEDEHYDRB-MONOMER,Clim0 (protease substrate),282.7,acnB
4,ACYLCOADEHYDROG-MONOMER,Clim0 (protease substrate),86.0,fadE
...,...,...,...,...
1907,YRAO-MONOMER,Clim0,333.7,diaA
1908,YRBF-MONOMER,Clim0,720.0,mlaF
1909,YTFQ-MONOMER,Clim0,718.3,ytfQ
1910,YTFR-MONOMER,Clim0,640.3,ytfR


In [90]:
# third, order the dataframe, then resplit the data 

# once again, split the dataframe: 
ordered_df = df_10hrs.sort_values(by="CLClim0NE Clim0 half life (min)", ascending=True)

# filter ordered DataFrame by HL Source using .loc[] method instead:
ordered_Clim_df = ordered_df.loc[ordered_df['HL Source'] == Clim_name]
ordered_Clim_PS_df = ordered_df.loc[ordered_df['HL Source'] == Clim_name + " (protease substrate)"]

,Protein ID,HL Source,CLClim0NE Clim0 half life (min),gene symbol
1227,G6964-MONOMER,Clim0,13.9,yeaC
803,EG11545-MONOMER,Clim0,15.9,flgL
245,RPOS-MONOMER,Clim0 (protease substrate),17.7,rpoS
196,G7596-MONOMER,Clim0 (protease substrate),22.7,patA
894,EG11967-MONOMER,Clim0,23.1,flgK
...,...,...,...,...
1585,MGTA-MONOMER,Clim0,720.0,mgtA
1221,G6949-MONOMER,Clim0,720.0,ynjB
393,CDPDIGLYPYPHOSPHA-MONOMER,Clim0,720.0,cdh
1598,MONOMER0-282,Clim0,720.0,phoE


In [107]:
# third, generate the graph
fig = go.Figure()

# generate the figure in the background first:
fig = px.scatter(ordered_df, x=ordered_df["Protein ID"], y=ordered_df["CLClim0NE Clim0 half life (min)"])
fig.update_traces(marker_size=.5, opacity=.1)

# add a second y-axis with the hour listed on it 
# fig.add_trace(go.Scatter(x=ordered_df['Protein ID'], y=(ordered_df['CLClim0NE Clim0 half life (min)']/60), mode='markers', marker_size=.01, opacity=.01, yaxis="y2", marker=dict(color="lightseagreen")))
# fig.update_layout(yaxis2=dict(title='Clim half life (hrs)', overlaying='y', side='right'))    

# overlay the individual rates
fig.add_trace(go.Scatter(x=(ordered_Clim_df['Protein ID']), y=(ordered_Clim_df["CLClim0NE Clim0 half life (min)"]), mode='markers', name=(Clim_name + ', '+ (str(np.shape(ordered_Clim_df['Protein ID'])[0]))), hovertext=ordered_Clim_df['gene symbol'], marker_size=3, marker=dict(color="lightseagreen"), yaxis="y1"))
fig.add_trace(go.Scatter(x=(ordered_Clim_PS_df['Protein ID']), y=(ordered_Clim_PS_df["CLClim0NE Clim0 half life (min)"]), mode='markers', name=(Clim_name + ' (protease substrates), ' + (str(np.shape(ordered_Clim_PS_df['Protein ID'])[0]))), hovertext=ordered_Clim_PS_df['gene symbol'], marker_size=5, marker=dict(color="orange", symbol='star'), yaxis="y1"))

# Plot Specs
fig.update_layout(title=f'The C-limited (Clim) half life values for the {str(np.shape(df_10hrs)[0])} proteins in the ' + current_sequence + ' PDR combo<br> that were assinged to a half life of 10 hours in the 2020 model (CLNE)', xaxis_title='Protein ID', yaxis_title='Clim Half Life (min)')
fig.update_xaxes(visible=False)
fig.update_layout(autosize=False, width=1200, height=700, showlegend=True)
fig.update_layout(legend_title_text='Half Life Source')

# wont open in pycharm, so save as a html:
out_pth = "~/wcEcoli/out/random_plotlys/" + current_sequence + "_Clim0_ordered_with_protein_substrates.html"
out_pth = os.path.expanduser(out_pth) 
output_dir = os.path.dirname(out_pth)
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it does not exist
fig.write_html(out_pth, auto_open=True) # displays the figure on a default web brouser 
